## Importing 

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from mpl_toolkits.mplot3d import Axes3D
import warnings
warnings.filterwarnings("ignore", module="matplotlib")
from sklearn.metrics import silhouette_score

from scipy.cluster.hierarchy import linkage, dendrogram
from scipy.cluster.hierarchy import fcluster
from sklearn.manifold import TSNE
from sklearn.model_selection import train_test_split

## Examing

In [ ]:
df = pd.read_csv("./data/bank-full.csv")
df.head()

In [ ]:
df.shape

In [ ]:
has_null = df.isnull().any().any()
has_null

In [ ]:
corr = df.corr()
plt.figure(figsize=(10, 8))
sns.heatmap(corr, annot=True, fmt=".2f", linewidths=.5, cmap='RdYlBu_r')
plt.show()

## Transformers

In [ ]:
def encode_categorical(df_orginal, reverse=False):
    df = df_orginal.copy(deep=True)
    
    months = ['jan', 'feb', 'mar', 'apr', 'may', 'jun', 'jul', 'aug', 'sep', 'oct', 'nov', 'dec']
    month_num = {months[i]: i+1 for i in range(len(months))}
    month_num_reverse = {i+1: months[i] for i in range(len(months))}
    
    edu_num = {'tertiary':2, 'secondary':1, 'unknown':-1, 'primary':0}
    edu_num_reverse = {v: k for k, v in edu_num.items()}
    
    marital_num = {'single':0, 'married':1, 'divorced':2}
    marital_num_reverse = {v: k for k, v in marital_num.items()}
    
    # Encoding for binary columns
    binary_num = {"no":0 ,"yes":1}
    binary_num_reverse = {v: k for k, v in binary_num.items()}
    
    binaries = [col for col in df.columns if set(df[col].unique()) in [{"yes", "no"}, {0, 1}]]

    if reverse:
        df["month"] = df["month"].map(month_num_reverse)
        df["education"] = df["education"].map(edu_num_reverse)
        df["marital"] = df["marital"].map(marital_num_reverse)
        df[binaries] = df[binaries].apply(lambda x: x.map(binary_num_reverse))
        
    else:
        df["month"] = df["month"].map(month_num)
        df["education"] = df["education"].map(edu_num)
        df["marital"] = df["marital"].map(marital_num)
        df[binaries] = df[binaries].apply(lambda x: x.map(binary_num))
    
    df.columns = df.columns.str.replace('.', ' ') 
    non_numeric_cols = df.select_dtypes(exclude=[np.number]).columns
    if len(non_numeric_cols) == 0:
        print("All columns numeric")
    else:
        print("Non-numeric columns are: ", non_numeric_cols)
    
    return df

In [ ]:
def calculate_woe(df, feature, target):
    total_events = df[target].sum()
    total_non_events = df.shape[0] - total_events

    groups = df.groupby(feature)[target].agg(['sum', 'count'])
    groups.columns = ['events', 'non_events']

    groups['events_rate'] = groups['events'] / total_events
    groups['non_events_rate'] = groups['non_events'] / total_non_events

    groups['woe'] = np.log(groups['events_rate'] / groups['non_events_rate'])
    groups['iv'] = (groups['events_rate'] - groups['non_events_rate']) * groups['woe']
    
    woe_dict = groups['woe'].to_dict()
    iv = groups['iv'].sum()
    
    return woe_dict, iv


In [ ]:
def encode_woe(df, cols=df.columns.tolist(), target="y"):
    df = df.copy(deep=True)
    ivs = {}
    woe_dicts = {}
    for col in cols:
        woe_dict, iv = calculate_woe(df, col, target)
        ivs[col] = iv
        woe_dicts[col] = woe_dict
        df[col] = df[col].apply(lambda x: woe_dict.get(x, 0))
        
    return df, woe_dicts, ivs

In [ ]:
def scaling(df, scaler=StandardScaler()):
    X_scaled = scaler.fit_transform(df)
    df_scaled = pd.DataFrame(X_scaled, columns=df.columns)
    return df_scaled, scaler

In [ ]:
df_e = encode_categorical(df)

In [ ]:
df_e.head()

In [ ]:
# instead of one-hot encoding to not extend dataframe
df_woe, woe_dicts, ivs = encode_woe(df_e, ["job", "poutcome", "contact"], "y")

In [ ]:
df_woe.head()

In [ ]:
non_hot = [col for col in df_woe.columns if "_" not in col]
df_woe[non_hot].boxplot(vert=False)
plt.show()

In [ ]:
df_scaled, scaler = scaling(df_woe)

In [ ]:
df_scaled

## K-means clustering method scaled data

In [ ]:
def pca(p_df):
    # Apply PCA
    pca = PCA(n_components=0.8)   # 0.8 means 80% variance
    principalComponents = pca.fit_transform(p_df)
    df_r = pd.DataFrame(principalComponents, columns = ['principal_' + str(i) for i in range(1, pca.n_components_ + 1)])

    # Check the number of components
    print("Number of components = ", pca.n_components_)

    # Check the explained variance ratio
    print("Explained variance ratio = ", sum(pca.explained_variance_ratio_))
    
    return df_r

In [ ]:
def count_wcss_scores(p_df, p_k):
    kmeans = KMeans(n_clusters = p_k, random_state = 0)
    kmeans.fit(p_df)
    return kmeans.score(p_df) * -1 # score returns -WCSS

In [ ]:
def count_clustering_scores(p_df, p_cluster_num, p_model, p_score_fun):
    if isinstance(p_cluster_num, int):
        cluster_num_iter = [p_cluster_num]
    else:
        cluster_num_iter = p_cluster_num
        
    scores = []    
    for k in cluster_num_iter:
        model_instance = p_model(n_clusters = k)
        labels = model_instance.fit_predict(p_df)
        wcss = p_score_fun(p_df, labels)
        scores.append(wcss)
    
    if isinstance(p_cluster_num, int):
        return scores[0]
    else:
        return scores

In [ ]:
def draw_elbow_wcss(p_df, p_k_max):
    cluster_num_seq = range(1, p_k_max)
    wcss_scores = count_clustering_scores(p_df, cluster_num_seq, KMeans, count_wcss_scores)
    plt.plot(cluster_num_seq, wcss_scores, 'bx-')
    plt.xlabel('k')
    plt.ylabel('wcss score')
    plt.title('The Elbow Method showing the optimal k')
    plt.show()

In [ ]:
def draw_elbow_shil(p_df, p_k_max):
    cluster_num_seq = range(2, p_k_max)
    silhouette_vec = count_clustering_scores(p_df, cluster_num_seq, KMeans, silhouette_score)
    plt.plot(cluster_num_seq, silhouette_vec, 'bx-')
    plt.xlabel('k')
    plt.ylabel('Silhouette score')
    plt.title('The Elbow Method showing the optimal k')
    plt.show()

In [ ]:
def cluster_KMeans(p_df, p_n_clusters):
    kmeans = KMeans(n_clusters = p_n_clusters, random_state = 0)

    # Fit the KMeans model to the data
    kmeans.fit(p_df)

    # Get the cluster assignments for each data point
    labels = kmeans.labels_

    # If you want to add the cluster labels back into your DataFrame
    p_df['cluster'] = labels

    print('Inertia:', kmeans.inertia_)

In [ ]:
df_principal = pca(df_scaled)

In [ ]:
# draw_elbow_wcss(df_principal, 50)

In [ ]:
# draw_elbow_shil(df_principal, 50)

In [ ]:
cluster_KMeans(df_principal, 13)
df_c = df.copy(deep = True)
df_c['cluster'] = df_principal['cluster']

## K-means clustering analysys

In [ ]:
fig = plt.figure(figsize=(10, 7))
ax = fig.add_subplot(111, projection='3d')

sc = ax.scatter(df_c['balance'], df_c['age'], df_c['duration'],   c=df_c['cluster'], alpha=0.6, edgecolors='w', s=80)

ax.set_xlabel('Balance')
ax.set_ylabel('Age')
ax.set_zlabel('Duration')

plt.colorbar(sc, label='cluster')  # Show color scale
plt.title('3D scatter plot of Age, Duration, and Balance')
plt.show()

In [ ]:
df_means = df_c.groupby('cluster')[['balance', 'age', 'duration']].median()
fig = plt.figure(figsize=(10, 7))
ax = fig.add_subplot(111, projection='3d')

# Get cluster labels and use as colors
clusters = df_means.index
colors = plt.cm.rainbow(np.linspace(0, 1, len(clusters)))

for cluster, color in zip(clusters, colors):
    ax.scatter(df_means.loc[cluster, 'balance'], df_means.loc[cluster, 'age'], df_means.loc[cluster, 'duration'], color=color, alpha=0.6, edgecolors='w', s=80, label='Cluster {}'.format(cluster))
    ax.text(df_means.loc[cluster, 'balance'], df_means.loc[cluster, 'age'], df_means.loc[cluster, 'duration'], '%s' % (str(cluster)), size=10, zorder=1, color='k')

ax.set_xlabel('Median Balance')
ax.set_ylabel('Median Age')
ax.set_zlabel('Median Duration')

plt.legend()
plt.title('3D scatter plot of median Age, Duration, and Balance per cluster')
plt.show()

In [ ]:
# Get the most common jobs for each cluster
top_jobs = df_c.groupby('cluster')['job'].apply(lambda x: x.value_counts().nlargest(3))

# Convert the multi-index series to a dataframe
top_jobs_df = top_jobs.reset_index()

# Rename columns
top_jobs_df.columns = ['cluster', 'job', 'count']

# View the DataFrame
top_jobs_df.head()

In [ ]:
import matplotlib.pyplot as plt

# Get unique jobs and clusters
unique_jobs = top_jobs_df['job'].unique()
unique_clusters = top_jobs_df['cluster'].unique()

# Set up the figure and axis
fig, ax = plt.subplots(figsize=(12, 8))

# Create a bar for each job
for idx, job in enumerate(unique_jobs):
    # Get the counts for each cluster for the current job
    counts = top_jobs_df[top_jobs_df['job'] == job]['count']
    clusters = top_jobs_df[top_jobs_df['job'] == job]['cluster']

    # Create a bar for each cluster
    for cluster_idx, count in enumerate(counts):
        # If this is the first cluster, create the base of the bar
        if cluster_idx == 0:
            bar = ax.bar(idx, count)
        # If this is not the first cluster, add to the existing bar
        else:
            bar = ax.bar(idx, count, bottom=counts.iloc[:cluster_idx].sum())

        # Add the cluster number to the bar segment
        height = counts.iloc[:cluster_idx+1].sum()
        ax.text(idx, height - count/2, f'CL:{clusters.iloc[cluster_idx]}', ha='center', va='center', color='white')

# Set up the x-axis
ax.set_xticks(range(len(unique_jobs)))
ax.set_xticklabels(unique_jobs, rotation=90)

# Add labels and title
ax.set_ylabel('Count')
ax.set_xlabel('Job')
ax.set_title('Cluster distribution for each job')

plt.show()

In [ ]:
cluster_details = {
    0: "old_low_balance_retired",
    1: "housemaid",
    2: "middle_low_balance",
    3: "young_low_balance",
    4: "admin_management",
    5: "long_duration",
    6: 6,
    7: 7,
    8: "students",
    9: 8,
    10: 10,
    11:"high_balance",
    12: 12,
}

## hierarchy clustering

In [ ]:
# my computer cannot operate on so many rows
df_reduced, _ = train_test_split(df, train_size=0.1, shuffle=True)

In [ ]:
df_reduced.head()

In [ ]:
df_reduced.shape

In [ ]:
def c_dendrogram(p_df, p_method):
    mergings = linkage(p_df, method=p_method)
    dendrogram(mergings, leaf_rotation=90, leaf_font_size=2)
    plt.show()
    
    return mergings

In [ ]:
def c_labels(p_mergings, p_fcluster_level):
    return fcluster(p_mergings, p_fcluster_level, criterion='distance')

In [ ]:
def c_tsne(p_df, columns):
    for columns in columns:
        print(f"COLUMN '{columns}'")
        model_2d = TSNE(learning_rate=200)
        transformed = model_2d.fit_transform(p_df)
        xs = transformed[:, 0]
        ys = transformed[:, 1]
        plt.scatter(xs, ys, c=p_df[columns])
        plt.show()

        model_3d = TSNE(n_components=3, learning_rate=200)
        transformed = model_3d.fit_transform(p_df)
        xs = transformed[:, 0]
        ys = transformed[:, 1]
        zs = transformed[:, 2]
        fig = plt.figure()
        ax = fig.add_subplot(projection='3d')
        sc = ax.scatter(xs, ys, zs, c=p_df[columns], alpha=0.6, edgecolors='w')
        plt.colorbar(sc, label='cluster')
        plt.show()
        print()


In [ ]:
df

In [ ]:
merg_complete = c_dendrogram(df_n, 'complete')

In [ ]:
df["labels_complete"] = c_labels(merg_complete, 10.5)

In [ ]:
merg_single = c_dendrogram(df_n, 'single')

In [ ]:
df["labels_single"] = c_labels(merg_single, 5.5)

In [ ]:
merg_average = c_dendrogram(df_n, 'average')

In [ ]:
df["labels_average"] = c_labels(merg_average, 7.75)

In [ ]:
merg_weighted = c_dendrogram(df_n, 'weighted')

In [ ]:
df["labels_weighted"] = c_labels(merg_weighted, 8)

In [ ]:
merg_centroid = c_dendrogram(df_n, 'centroid')

In [ ]:
df["labels_centroid"] = c_labels(merg_centroid, 6)

In [ ]:
merg_median = c_dendrogram(df_n, 'median')

In [ ]:
df_scaled["labels_median"] = c_labels(merg_median, 6)

In [ ]:
merg_ward = c_dendrogram(df_n, 'ward')

In [ ]:
df_scaled["labels_ward"] = c_labels(merg_ward, 15)

In [ ]:
c_tsne(df_scaled, ["labels_complete",
                   "labels_single",
                   "labels_average",
                   "labels_weighted",
                   "labels_centroid",
                   "labels_median", 
                   "labels_ward"])

In [ ]:
labels = [column for column in df_scaled.columns if column.startswith("labels_")]

In [ ]:
for label in labels:
    print(f'in "{label}" column are {len(df_scaled[label].unique())} clusters')